### Import librairies

In [1]:
import os
import pandas as pd
import anndata as ad
from FlowCytometryTools import FCMeasurement
import re
import numpy as np
#from list_paths import create_list_of_paths,create_list_of_paths_spec_patients
from plot_pred_vs_true import plot_result

### Create functions to load and concatenate data

In [2]:
def arcsinh_transform(X,cofactor=5):
    return np.arcsinh(X/cofactor)

In [89]:
def normalize_by_control(X, control_p75):
    X_norm = X / control_p75
    return np.log1p(X_norm)

In [19]:
def extract_patient_from_path(file_path):
    """extract patient id from file name (works for DOMS, to adapts for others)"""
    file_name = os.path.basename(file_path)
    file_name_without_ext = os.path.splitext(file_name)[0]
    #patient_id = file_name_without_ext.split("_")[4]+'_'+file_name_without_ext.split("_")[5]
    #patient_id = file_name_without_ext.split("-")[0]
    patient_id = file_name_without_ext.split("_")[1]
    return patient_id

In [90]:
def load_data_fcs_raw(path, condition_name, cell_type=None):
    data = FCMeasurement(ID="Sample", datafile=path)
    anndata = ad.AnnData(data.data)
    anndata.obs_names_make_unique()
    anndata.obs['drug'] = condition_name

    if cell_type:
        anndata.obs['cell_type'] = cell_type

    patient_id = extract_patient_from_path(path)
    anndata.obs['patient'] = patient_id

    return anndata


In [92]:
def concatenate_2conditions_multiple_data_raw(path_stim_list, stim_name, path_unstim_list, unstim_name, outdir_path):
    unstim_list = [load_data_fcs_raw(path, unstim_name, cell_type) for path, cell_type in path_unstim_list]
    stim_list = [load_data_fcs_raw(path, stim_name, cell_type) for path, cell_type in path_stim_list]

    comb_unstim_anndata = ad.concat(unstim_list, join='outer', axis=0)
    control_p75 = np.percentile(comb_unstim_anndata.X, 75, axis=0)
    control_p75[control_p75 == 0] = 1e-6
    comb_unstim_anndata.X = normalize_by_control(comb_unstim_anndata.X, control_p75)
    for i in range(len(stim_list)):
        stim_list[i].X = normalize_by_control(stim_list[i].X, control_p75)

    comb_stim_anndata = ad.concat(stim_list, join='outer', axis=0)
    comb_stim_anndata.var = comb_stim_anndata.var.astype(str)
    comb_unstim_anndata.var = comb_unstim_anndata.var.astype(str)

    combined_anndata = ad.concat([comb_unstim_anndata, comb_stim_anndata], join='outer', axis=0)

    directory = os.path.dirname(outdir_path)
    os.makedirs(directory, exist_ok=True)
    combined_anndata.write_h5ad(outdir_path)

    return

In [4]:
def load_data_fcs(path,condition_name,cell_type=None):
    data = FCMeasurement(ID="Sample", datafile=path)
    anndata = ad.AnnData(data.data)
    anndata.obs_names_make_unique()
    anndata.obs['drug']= condition_name
    
    if cell_type:
        anndata.obs['cell_type']= cell_type
    patient_id = extract_patient_from_path(path)
    anndata.obs['patient'] = patient_id
    anndata.X=arcsinh_transform(anndata.X)
    return anndata

In [51]:
def concatenate_2conditions_data(path_stim,stim_name,path_unstim,unstim_name,outdir_path):
    unstim_anndata = load_data_fcs(path_unstim,unstim_name)
    stim_anndata = load_data_fcs(path_stim,stim_name)

    combined_anndata = stim_anndata.concatenate(unstim_anndata, batch_key="condition", batch_categories=["stim", "control"])
    combined_anndata.write(outdir_path)
    return

In [93]:
def concatenate_2conditions_multiple_data(path_stim_list,stim_name,path_unstim_list,unstim_name,outdir_path):
    comb_unstim_anndata = load_data_fcs(path_unstim_list[0][0],unstim_name,path_unstim_list[0][1])
    comb_stim_anndata = load_data_fcs(path_stim_list[0][0],stim_name,path_stim_list[0][1])

    for path_stim in path_stim_list[1:]:
        if path_stim[1]:
            stim_anndata = load_data_fcs(path_stim[0],stim_name,path_stim[1])
        else:
            stim_anndata = load_data_fcs(path_stim[0],stim_name)
        assert isinstance(comb_stim_anndata, ad.AnnData), f"Expected AnnData, got {type(comb_stim_anndata)}"
        assert isinstance(stim_anndata, ad.AnnData), f"Expected AnnData, got {type(stim_anndata)}"
        comb_stim_anndata.obs.reset_index(drop=True, inplace=True)
        stim_anndata.obs.reset_index(drop=True, inplace=True)

        # Concaténer les deux objets AnnData
        comb_stim_anndata = ad.concat([comb_stim_anndata, stim_anndata], join='outer', axis=0)
        #comb_stim_anndata = comb_stim_anndata.concatenate(stim_anndata)
        
    for path_unstim in path_unstim_list[1:]:
        if path_unstim[1]:
            unstim_anndata = load_data_fcs(path_unstim[0],unstim_name,path_unstim[1])
        else:
            unstim_anndata = load_data_fcs(path_unstim[0],unstim_name)
        comb_unstim_anndata = ad.concat([comb_unstim_anndata, unstim_anndata], join='outer', axis=0)
        #comb_unstim_anndata = comb_unstim_anndata.concatenate(unstim_anndata)
    comb_stim_anndata.var = comb_stim_anndata.var.astype(str)
    comb_unstim_anndata.var = comb_unstim_anndata.var.astype(str)

    combined_anndata = ad.concat([comb_unstim_anndata, comb_stim_anndata], join='outer', axis=0)
    #combined_anndata.write(outdir_path)
    directory=os.path.dirname(outdir_path)
    os.makedirs(directory, exist_ok=True)
    combined_anndata.write_h5ad(outdir_path)

    return 

In [8]:
def concatenate_1condition_multiple_data(path_list,condition_name,outdir_path):
    comb_anndata = load_data_fcs(path_list[0],condition_name)

    for path in path_list[1:]:
        new_anndata = load_data_fcs(path,condition_name)
        comb_anndata = comb_anndata.concatenate(new_anndata)
    comb_anndata.write(outdir_path)
    return 

In [6]:
def create_list_of_paths2(
    directory, stimulation, cell_type=None, sample=None, patient_excluded=[]
):
    paths_list = []
    for filename in os.listdir(directory):
        if len(patient_excluded) > 0:
            for patient in patient_excluded:
                if (
                    stimulation in filename
                    and (cell_type is None or cell_type in filename)
                    and (sample is None or sample in filename)
                    and patient not in filename
                ):
                    paths_list.append([os.path.join(directory, filename),cell_type])
        else:
            if (
                stimulation in filename
                and (cell_type is None or cell_type in filename)
                and (sample is None or sample in filename)
            ):
                paths_list.append([os.path.join(directory, filename),cell_type])
    return paths_list

In [7]:
def create_list_of_paths_spec_patients(
    directory, stimulation, cell_type=None, sample=None, patient=None
):
    paths_list = []
    for filename in os.listdir(directory):
        if (
            stimulation in filename
            and (cell_type is None or cell_type in filename)
            and (sample is None or sample in filename)
            and (patient is None or patient in filename)
        ):
            paths_list.append([os.path.join(directory, filename),cell_type])
    return paths_list

In [57]:
def concatenate_2cohorts(path_coh_1_list,path_coh_2_list,outdir_path_list,cohort_1,cohort_2):
    directory=os.path.dirname(outdir_path_list[0])
    os.makedirs(directory, exist_ok=True)
    for path_coh1,path_coh2,outdir_path in zip(path_coh_1_list,path_coh_2_list,outdir_path_list):
        data1=ad.read_h5ad(path_coh1)
        data2=ad.read_h5ad(path_coh2)
        data1.obs['cohort']= cohort_1
        data2.obs['cohort']= cohort_2
        data1.obs['drug'] = data1.obs['drug'].replace('P. gingivalis', 'LPS')
        cell_type= data1.obs['cell_type'].unique()[0]
        data2.obs['cell_type'] = cell_type
        comb_stim_anndata = ad.concat([data1, data2], join='outer', axis=0)
        comb_stim_anndata.var = comb_stim_anndata.var.astype(str)
        comb_stim_anndata.obs.reset_index(drop=True, inplace=True)
        
        comb_stim_anndata.write_h5ad(outdir_path)
    return 

###  Examples

In [22]:
perio_stim_list=['IFNa','LPS']
perio_cell_list=['Granulocytes (CD45-CD66+)','B-Cells (CD19+CD3-)','Classical Monocytes (CD14+CD16-)','MDSCs (lin-CD11b-CD14+HLADRlo)','mDCs (CD11c+HLADR+)','pDCs(CD123+HLADR+)','Intermediate Monocytes (CD14+CD16+)','Non-classical Monocytes (CD14-CD16+)','CD56+CD16- NK Cells','CD56loCD16+NK Cells','NK Cells (CD7+)','CD4 T-Cells','Tregs (CD25+FoxP3+)','CD8 T-Cells','CD8-CD4- T-Cells']

In [85]:
cohort_1='perio'
cohort_2='doms'
path_coh_1_list=['perio_just_concat/'+f for f in os.listdir('perio_just_concat') if 'TNFa' not in f]
path_coh_2_list=['doms_just_concat/'+f for f in os.listdir('doms_just_concat') if 'TNFa' not in f]
path_coh_1_list.sort()
path_coh_2_list.sort()
outdir_path_list=[f'{cohort_1}_{cohort_2}_just_concat/{f.split("_")[2]}_{f.split("_")[3]}' for f in os.listdir('doms_just_concat')]
outdir_path_list.sort()

In [ ]:
c=0
tot=len(perio_stim_list)*len(perio_cell_list)
for cell_type in perio_cell_list:
    for stim in perio_stim_list:
        cell_doms=correspondence_doms[cell_type]
        path_unstim_list=create_list_of_paths_spec_patients(directory='DOMS_Helios_012025 - Gated Populations',stimulation='Unstim',cell_type=cell_doms)
        path_stim_list=create_list_of_paths_spec_patients(directory='DOMS_Helios_012025 - Gated Populations',stimulation=stim,cell_type=cell_doms)
        if len(path_stim_list)==0:
            print('no stim for ',cell_type,stim)
            assert False, f"no stim for {cell_type} {stim}"
            continue
        if len(path_unstim_list)==0:
            print('no unstim for ',cell_type,stim)
            assert False, f"no unstim for {cell_type} {stim}"
            continue
        res=concatenate_2conditions_multiple_data(path_stim_list,stim,path_unstim_list,'Unstim',"doms_just_concat/doms_data_"+stim+"_"+cell_type+".h5ad")
        c+=1
        print('done ',c,'/',tot)

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
... storing 'cell_type' as categorical
... storing 'patient' as categorical
... storing 'cohort' as categorical
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
... storing 'cell_type' as categorical
... storing 'patient' as categorical
... storing 'cohort' as categorical
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
..

In [21]:
correspondence_doms = {}
correspondence_doms['Granulocytes (CD45-CD66+)'] ='Neutrophils'
correspondence_doms['B-Cells (CD19+CD3-)'] ='B-cells'
correspondence_doms['Classical Monocytes (CD14+CD16-)'] ='cMCs'
correspondence_doms['MDSCs (lin-CD11b-CD14+HLADRlo)'] ='MDSCs'
correspondence_doms['mDCs (CD11c+HLADR+)'] ='mDCs'
correspondence_doms['pDCs(CD123+HLADR+)'] ='pDCs'
correspondence_doms['Intermediate Monocytes (CD14+CD16+)'] ='intMCs'
correspondence_doms['Non-classical Monocytes (CD14-CD16+)'] ='ncMCs'
correspondence_doms['CD56+CD16- NK Cells'] ='CD16- CD56+ NK'
correspondence_doms['CD56loCD16+NK Cells'] ='CD16+ CD56lo NK'
correspondence_doms['NK Cells (CD7+)'] ='NK'
correspondence_doms['CD4 T-Cells'] ='CD4 T-cells'
correspondence_doms['Tregs (CD25+FoxP3+)'] ='Tregs'
correspondence_doms['CD8 T-Cells'] ='CD8 T-cells'
correspondence_doms['CD8-CD4- T-Cells'] ='CD4- CD8- T-cells'


In [14]:
correspondence_surge = {}
correspondence_surge['Granulocytes (CD45-CD66+)'] ='granulocytes'
correspondence_surge['B-Cells (CD19+CD3-)'] ='Bcells'
correspondence_surge['Classical Monocytes (CD14+CD16-)'] ='cMC'
correspondence_surge['MDSCs (lin-CD11b-CD14+HLADRlo)'] ='MDSC'
correspondence_surge['mDCs (CD11c+HLADR+)'] ='mDC'
correspondence_surge['pDCs(CD123+HLADR+)'] ='pDC'
correspondence_surge['Intermediate Monocytes (CD14+CD16+)'] ='intMC'
correspondence_surge['Non-classical Monocytes (CD14-CD16+)'] ='ncMc'
correspondence_surge['CD56+CD16- NK Cells'] ='CD56brightCD16negNK'
correspondence_surge['CD56loCD16+NK Cells'] ='CD56dimCD16posNK'
correspondence_surge['NK Cells (CD7+)'] ='NK'
correspondence_surge['CD4 T-Cells'] ='CD4Tcells'
correspondence_surge['Tregs (CD25+FoxP3+)'] ='Treg'
correspondence_surge['CD8 T-Cells'] ='CD8Tcells'
correspondence_surge['CD8-CD4- T-Cells'] ='CD4negCD8neg'

In [23]:
c=0
tot=len(perio_stim_list)*len(perio_cell_list)
for cell_type in perio_cell_list:
    for stim in perio_stim_list:
        cell_doms=correspondence_doms[cell_type]
        path_unstim_list=create_list_of_paths_spec_patients(directory='DOMS_Helios_012025 - Gated Populations',stimulation='Unstim',cell_type=cell_doms)
        path_stim_list=create_list_of_paths_spec_patients(directory='DOMS_Helios_012025 - Gated Populations',stimulation=stim,cell_type=cell_doms)
        if len(path_stim_list)==0:
            print('no stim for ',cell_type,stim)
            assert False, f"no stim for {cell_type} {stim}"
            continue
        if len(path_unstim_list)==0:
            print('no unstim for ',cell_type,stim)
            assert False, f"no unstim for {cell_type} {stim}"
            continue
        res=concatenate_2conditions_multiple_data(path_stim_list,stim,path_unstim_list,'Unstim',"doms_just_concat/doms_data_"+stim+"_"+cell_type+".h5ad")
        c+=1
        print('done ',c,'/',tot)

/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", I

done  1 / 30


/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificat

done  2 / 30
done  3 / 30


/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Observation names are not unique. To make them unique, ca

...

done  24 / 30
done  25 / 30
done  26 / 30


/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificat

done  27 / 30
done  28 / 30


/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", I

done  29 / 30
done  30 / 30


In [57]:
surge_stim_list=['LPS','TNFa']
surge_cell_list=['Granulocytes (CD45-CD66+)','B-Cells (CD19+CD3-)','Classical Monocytes (CD14+CD16-)','MDSCs (lin-CD11b-CD14+HLADRlo)','mDCs (CD11c+HLADR+)','pDCs(CD123+HLADR+)','Intermediate Monocytes (CD14+CD16+)','Non-classical Monocytes (CD14-CD16+)','CD56+CD16- NK Cells','CD56loCD16+NK Cells','NK Cells (CD7+)','CD4 T-Cells','Tregs (CD25+FoxP3+)','CD8 T-Cells','CD8-CD4- T-Cells']

In [70]:
markers_doms=['149Sm_pCREB','150Nd_pSTAT5','151Eu_pp38','153Eu_pSTAT1','154Sm_pSTAT3','155Gd_pS6','159Tb_pMAPKAPK2','167Er_pERK','168Er_pSTAT6','169Tm_CD25','174Yb_HLADR','164Dy_IkB','166Er_pNFkB']

In [58]:
correspondence = {}
correspondence['Granulocytes (CD45-CD66+)'] ='granulocytes'
correspondence['B-Cells (CD19+CD3-)'] ='Bcells'
correspondence['Classical Monocytes (CD14+CD16-)'] ='cMC'
correspondence['MDSCs (lin-CD11b-CD14+HLADRlo)'] ='MDSC'
correspondence['mDCs (CD11c+HLADR+)'] ='mDC'
correspondence['pDCs(CD123+HLADR+)'] ='pDC'
correspondence['Intermediate Monocytes (CD14+CD16+)'] ='intMC'
correspondence['Non-classical Monocytes (CD14-CD16+)'] ='ncMc'
correspondence['CD56+CD16- NK Cells'] ='CD56brightCD16negNK'
correspondence['CD56loCD16+NK Cells'] ='CD56dimCD16posNK'
correspondence['NK Cells (CD7+)'] ='NK'
correspondence['CD4 T-Cells'] ='CD4Tcells'
correspondence['Tregs (CD25+FoxP3+)'] ='Treg'
correspondence['CD8 T-Cells'] ='CD8Tcells'
correspondence['CD8-CD4- T-Cells'] ='CD4negCD8neg'

# Affichage du dictionnaire obtenu
print(correspondence)

{'Granulocytes (CD45-CD66+)': 'granulocytes', 'B-Cells (CD19+CD3-)': 'Bcells', 'Classical Monocytes (CD14+CD16-)': 'cMC', 'MDSCs (lin-CD11b-CD14+HLADRlo)': 'MDSC', 'mDCs (CD11c+HLADR+)': 'mDC', 'pDCs(CD123+HLADR+)': 'pDC', 'Intermediate Monocytes (CD14+CD16+)': 'intMC', 'Non-classical Monocytes (CD14-CD16+)': 'ncMc', 'CD56+CD16- NK Cells': 'CD56brightCD16negNK', 'CD56loCD16+NK Cells': 'CD56dimCD16posNK', 'NK Cells (CD7+)': 'NK', 'CD4 T-Cells': 'CD4Tcells', 'Tregs (CD25+FoxP3+)': 'Treg', 'CD8 T-Cells': 'CD8Tcells', 'CD8-CD4- T-Cells': 'CD4negCD8neg'}


In [71]:
ad.read('pred_surge_corrected (4).h5ad').obs

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


,drug,cell_type,patient,stim,state
0,TNFa,intMC,HCFF_BL,TNFa,predicted
1,TNFa,intMC,HCFF_BL,TNFa,predicted
2,TNFa,intMC,HCFF_BL,TNFa,predicted
3,TNFa,intMC,HCFF_BL,TNFa,predicted
4,TNFa,intMC,HCFF_BL,TNFa,predicted
...,...,...,...,...,...
1387,TNFa,intMC,HCKK_BL,TNFa,true_corrected
1388,TNFa,intMC,HCKK_BL,TNFa,true_corrected
1389,TNFa,intMC,HCKK_BL,TNFa,true_corrected
1390,TNFa,intMC,HCKK_BL,TNFa,true_corrected


In [59]:
c=0
tot=len(surge_stim_list)*len(surge_cell_list)
for cell_type in surge_cell_list:
    surge_name=correspondence[cell_type]
    for stim in surge_stim_list:
        path_unstim_list=create_list_of_paths2(directory='surge_prehab_controls - Gated Populations',stimulation='nstim',cell_type=surge_name)
        path_stim_list=create_list_of_paths2(directory='surge_prehab_controls - Gated Populations',stimulation=stim,cell_type=surge_name)
        half=len(path_stim_list)//2
        if len(path_stim_list)==0:
            print(cell_type,stim)
            break
        path_stim_list_train=path_stim_list[:half]
        path_stim_list_test=path_stim_list[half:]
        path_unstim_list_train=path_unstim_list[:half]
        path_unstim_list_test=path_unstim_list[half:]
        
        restrain=concatenate_2conditions_multiple_data(path_stim_list_train,stim,path_unstim_list_train,'Unstim',"surge_concatenated_train/surge_concatenated_"+stim+"_"+cell_type+".h5ad")
        restest=concatenate_2conditions_multiple_data(path_stim_list_test,stim,path_unstim_list_test,'Unstim',"surge_concatenated_test/surge_concatenated_"+stim+"_"+cell_type+".h5ad")
        c+=1
        print('done ',c,'/',tot)


/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", I

done  1 / 30


/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", I

done  2 / 30


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
... storing 'drug' as categorical
... storing 'cell_type' as categorical
... storing 'patient' as categorical
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/olivierf/stanford/cellot/c

...

done  28 / 30


/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Observation names are not unique. To make them unique, ca

done  29 / 30


/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
/Users/olivierf/stanford/cellot/cellot_pregnancy/create_data_cellot/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Observation names are not unique. To make them unique, ca

done  30 / 30


In [9]:
path_unstim_pred=create_list_of_paths_spec_patients(directory='datasets/PTB drug screen complete - Gated Populations',stimulation='Unstim',cell_type='cMC',patient='HV01')
path_stim_pred=create_list_of_paths_spec_patients(directory='datasets/PTB drug screen complete - Gated Populations',stimulation='LPS',cell_type='cMC',patient='HV01')
res=concatenate_2conditions_multiple_data(path_stim_pred,'LPS',path_unstim_pred,'Unstim',"datasets/PTB_training/LPS_cMC_HV01.h5ad")

/Users/MacBook/miniconda3/envs/cellot_env/lib/python3.8/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
... storing 'drug' as categorical
... storing 'cell_type' as categorical


### Pipeline for evaluation

In [2]:
EVALUATION_MARKERS = [
    '151Eu_pp38', '155Gd_pS6', '149Sm_pCREB', 
    '159Tb_pMAPKAPK2', '166Er_pNFkB', '167Er_pERK12'
]

In [ ]:
PTB_EVALUATION_MARKERS = ['151Eu_pP38', '155Gd_pS6', '149Sm_pCREB', '159Tb_pMK2', '166Er_pNFkB', '167Er_pERK']

train on CMC, pp38, pS6, LPS

In [148]:
original_path = "datasets/PTB_training/combined_LPS_cMC.h5ad"
target = ad.read(original_path)
target1 = target[:, '151Eu_pP38'].copy()

In [154]:
target1[target1.obs['condition'] == 'control'].X.flatten()

ArrayView([2.0405424, 1.9779768, 0.6029839, ..., 0.8639047, 0.902362 ,
           1.681758 ], dtype=float32)

In [ ]:
for marker in PTB_EVALUATION_MARKERS:
    plot_result('results/PTB/LPS_cMC/model-cellot/pred.csv','datasets/PTB_training/LPS_cMC_HV01.h5ad',marker,f"results/PTB/LPS_cMC/model-cellot/unseen_plot/LPS_cMC_HV01_{marker}.png")

In [101]:
np.mean(ad.read('unstim_perio/perio_data_sherlockIFNa_B-Cells (CD19+CD3-)_PO1.h5ad')[:,'159Tb_MAPKAPK2'][ad.read('unstim_perio/perio_data_sherlockIFNa_B-Cells (CD19+CD3-)_PO1.h5ad')[:,'159Tb_MAPKAPK2'].obs['drug']=='Unstim'].X.flatten())

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


ArrayView(2.0665224, dtype=float32)

In [103]:
plot_result('./pred.csv','perio_data_sherlock2/perio_data_sherlockIFNa_B-Cells (CD19+CD3-)_train.h5ad','159Tb_MAPKAPK2','./test_perio.png','IFNa')

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [30]:
target = ad.read('surge_corr_LPS_mDCs (CD11c+HLADR+).h5ad')
marker = '149Sm_pCREB'
stim = 'LPS'
target1 = target[:, marker].copy()
stim = pd.Series(
    target1[target1.obs["drug"] == stim].X.flatten(), name="Stim True"
)
unstim = pd.Series(
    target1[target1.obs["drug"] == "Unstim"].X.flatten(), name="Unstim"
)
dataf = ad.read('surge_concatenated/surge_concatenated_LPS_mDCs (CD11c+HLADR+).h5ad')
dataf = dataf[:, marker].copy()
stim2 = pd.Series(
    dataf[dataf.obs["drug"] == 'Unstim'].X.flatten(), name="Stim Pred"
)
dist_data = {
    "Patient_1": {
        "Stim True": stim.values,
        "Stim Pred": stim2.values,
        "Unstim": unstim.values,
    }
}
from plot_pred_vs_true import create_density_plots
create_density_plots(dist_data, 'unstim.png', title_suffix="")

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
